# Authenticating To LightTag's API

LightTag will provision a temproary API token when you use the API. This section shows how to provision that token and configure a requests session for simple use

When using the LightTag API, copy this code and use the session object we create

In [22]:
import requests 
class LTSession(requests.Session):
    def __init__(self, workspace:str,user:str,pwd:str,host:str=None):
        """[summary]
        Initializes a LightTag API session object
        Arguments:
            workspace {str} -- [The name of your workspace]
            user {str} -- [Your LightTag Username]
            pwd {str} -- [Your LightTag password]
        
        Keyword Arguments:
            host {str} -- [Optional, use this when self hosting LightTag ] (default: {None})
        """
        super().__init__()
        if host is None:
            host = 'https://{workspace}.lighttag.io/api/'.format(workspace=workspace)
        token =self._authenticate(host,user,pwd)
        self.headers.update({"Authorization":"Token {token}".format(token=token)})
        
    @staticmethod
    def _authenticate(host:str,user:str,pwd:str):
        """Fetches authentication token based on username password
        
        Arguments:
            host {str} -- [Fully qualified host name]
            user {str} -- [LightTag Username]
            pwd {str} -- [LightTag Password]
        """
        response = requests.post(host+'auth/token/login/',json={"username":user,"password":pwd})
        response.raise_for_status()
        data = response.json()
        token = data['key']
        is_manager = data['is_manager']
        if not is_manager:
            raise Exception("Your are not a manager")
        return token


